In [3]:
import gc,uuid
import pandas as pd
import numpy as np
import pyarrow as pa
import tensorflow as tf
from pyarrow import parquet as pq
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder

In [4]:
train = pd.read_csv('../Data/Train/sales_train_validation.csv')
train = train.drop(['item_id','cat_id','store_id'],axis =1)
# train = train.drop(['item_id','dept_id','cat_id','store_id','state_id'],axis =1)
train.index = train.id
train = train.drop('id',axis = 1).head(10)

In [5]:
testDate = list(map(lambda x: 'd_'+str(x),list(range(1913-200,1913))))
test = train[['dept_id','state_id']+testDate]
train = train.drop(testDate,axis = 1)

In [6]:
# To-Do
# 1. Calendar event generator corresponding to certain date
# 2. Sell Price generator corresponding to certain date
# 4. Make tensorflow dataset

class feature_engineering(object):
    
    def __init__ (self,df,dimList,encoder = LabelEncoder,encodeDict = None):
        super().__init__()
        self._df = df
        self._dimList = dimList
        self.arr = df.drop(dimList,axis =1).values
        self.indexList = df.index.tolist()
        self._encoder = encoder
        self.labelDict,self.encodeDict = self._pandas_to_categorical_encode(encodeDict)


    def _rolling_window(self,a, window):
        shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
        strides = a.strides + (a.strides[-1],)
        return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)

    def _get_time_tensor(self,arr,window_size):
        tmp = self._rolling_window(arr,window_size+1)
        Xtensor = tmp[:,:-1].reshape(-1,window_size,1)
        Ytensor = tmp[:,-1].reshape(-1,1)
        return (Xtensor,Ytensor)

    def _tensor_factory(self,arr,window_size,categoryIx):
        X,Ytensor = self._get_time_tensor(arr,window_size)
        Xtensor = {}
        for i in self.labelDict:
            label = self.labelDict[i][categoryIx]
            Xtensor[i] = self._label_shape_transform(label,Ytensor.shape)
        Xtensor['sells'] = X
        return (Xtensor,Ytensor)

    def np_to_time_tensor_generator(self,windowSize):
        if np.ndim(self.arr) > 1:
            for ix,v in enumerate(self.arr):
                yield self._tensor_factory(v,windowSize,ix)
        else:
            yield self._tensor_factory(self.arr,windowSize,0) 

    def _label_encode(self,arr,encoder):
        if encoder is None:
            encoder = self._encoder()
            enc_arr = encoder.fit_transform(arr)
        else:
            enc_arr = encoder.transform(arr)
        return enc_arr,encoder

    def _pandas_to_categorical_encode(self,encodeDict):
        if encodeDict is None:
            encodeDict = {}
        labelDict = {}
        for i in self._dimList:
            if i in encodeDict:
                enc_arr,encoder = self._label_encode(self._df[i],encodeDict[i])
            else:
                enc_arr,encoder = self._label_encode(self._df[i],None)
            encodeDict[i] = encoder
            labelDict[i] = enc_arr
        return labelDict,encodeDict

    def _label_shape_transform(self,label,shape):
        tmp = np.zeros(shape)
        tmp += label
        return tmp

    def get_encoder_class(self,label):
        return len(self.encodeDict[label].classes_)



# Model Training

In [7]:
fe = feature_engineering(train,['dept_id','state_id'])
dept_id_class_num = fe.get_encoder_class('dept_id')
state_id_class_num = fe.get_encoder_class('state_id')
windowSize = 100

cacheFile = '../Data/Train/cache/'+str(uuid.uuid4())
train_univariate = tf.data.Dataset.from_generator(
    fe.np_to_time_tensor_generator,
    (
        {'sells':tf.float32, 'dept_id':tf.int16,'state_id':tf.int16},
        tf.float32
    ),
    output_shapes = (
        {
            'sells':tf.TensorShape([None,windowSize,1]),
            'dept_id':tf.TensorShape([None,1]),
            'state_id':tf.TensorShape([None,1])},
        tf.TensorShape([None,1])
        ),
    args = [windowSize])
train_univariate = train_univariate.prefetch(tf.data.experimental.AUTOTUNE).cache().repeat()

In [8]:
ge = feature_engineering(train,['dept_id','state_id'],encodeDict= fe.encodeDict)

cacheFile = '../Data/Val/cache/'+str(uuid.uuid4())
val_univariate = tf.data.Dataset.from_generator(
    ge.np_to_time_tensor_generator,
    (
        {'sells':tf.float32, 'dept_id':tf.int16,'state_id':tf.int16},
        tf.float32
    ),
    output_shapes = (
        {
            'sells':tf.TensorShape([None,windowSize,1]),
            'dept_id':tf.TensorShape([None,1]),
            'state_id':tf.TensorShape([None,1])},
        tf.TensorShape([None,1])
        ),
    args = [windowSize])
vals_univariate = val_univariate.prefetch(tf.data.experimental.AUTOTUNE).cache().repeat()

In [9]:
sells_input = tf.keras.layers.Input(shape=(windowSize,1),name = 'sells')
dept_id_input = tf.keras.layers.Input(shape=1,name = 'dept_id')
state_id_input = tf.keras.layers.Input(shape=1,name = 'state_id')

dept_embed = tf.keras.layers.Embedding(dept_id_class_num,3)(dept_id_input)
dept_embed =tf.keras.layers.Flatten()(dept_embed)

lstm = tf.keras.layers.LSTM(29)(sells_input)
dense = tf.keras.layers.Concatenate()([lstm,dept_embed])
dense = tf.keras.layers.Dense(10,'elu')(dense)
dense = tf.keras.layers.Dense(1)(dense)
simple_lstm_model = tf.keras.models.Model({'sells':sells_input,'dept_id':dept_id_input,'state_id':state_id_input},dense)
simple_lstm_model.compile(optimizer='adam', loss='mae')

In [10]:
EVALUATION_INTERVAL = 2000
EPOCHS = 3

simple_lstm_model.fit(
    train_univariate,
    epochs=EPOCHS,
    steps_per_epoch= EVALUATION_INTERVAL,
    validation_data=vals_univariate, 
    validation_steps=50
    )

Train for 2000 steps, validate for 50 steps
Epoch 1/3
2000/2000 [==============================] - 213s 106ms/step - loss: 1.1326 - val_loss: 1.1073
Epoch 2/3
2000/2000 [==============================] - 204s 102ms/step - loss: 1.1003 - val_loss: 1.0917
Epoch 3/3
2000/2000 [==============================] - 205s 103ms/step - loss: 1.0855 - val_loss: 1.0775
